In [1]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.chdir(u'D:/工作文件/氣象局計畫/110計畫/災損資料/氣象測站資料/201601-202004/2016-2020cwb_hr/raw_data')
os.getcwd()

'D:\\工作文件\\氣象局計畫\\110計畫\\災損資料\\氣象測站資料\\201601-202004\\2016-2020cwb_hr\\raw_data'

### 讀取原始測站txt檔案
處理column讀取後跑掉問題  
迴圈化後可以完整讀取所有檔案

### 測站資訊
先根據測站資訊選擇需要的測站
再進行氣象資料篩選(直接讀取完整氣象資料記憶體不足)

In [21]:
st_info = pd.read_csv(u'D:/工作文件/氣象局計畫/110計畫/災損資料/氣象測站資料/201601-202004/2016-2020cwb_hr/stinfo.csv')

In [22]:
st_info

,站號,站名,海拔高度(m),經度,緯度,城市,地址,地區,資料起始日期
0,466850,五分山雷達站,756.0,121.781205,25.071182,新北市,瑞芳區靜安路四段1巷1號,北,1998/7/1
1,466880,板橋,9.7,121.442017,24.997647,新北市,板橋區大觀路二段265巷62號,北,1972/3/1
2,466900,淡水,19.0,121.448906,25.164889,新北市,淡水區中正東路42巷6號,北,1942/1/1
3,466910,鞍部,837.6,121.529731,25.182586,臺北市,北投區陽明山竹子湖路111號,北,1937/1/1
4,466920,臺北,5.3,121.514853,25.037658,臺北市,中正區公園路64號,北,1896/01/01
...,...,...,...,...,...,...,...,...,...
609,C1Z110,三棧,680.0,121.605010,24.115570,花蓮縣,秀林鄉雙露山,東,2017/12/1
610,C1Z120,壽豐,62.0,121.508081,23.870853,花蓮縣,壽豐鄉壽豐村壽山路37號(壽豐鄉文康中心),東,2017/6/3
611,C1Z130,銅門,187.0,121.492842,23.965694,花蓮縣,秀林鄉銅門村69號(銅門國小),東,2017/6/3
612,C1Z140,荖溪,950.0,121.459640,23.890575,花蓮縣,秀林鄉,東,2018/3/23


In [89]:
county_list = list(set(list(st_info['城市'])))
county = dict(zip(county_list,list(np.arange(0,len(county_list)))))
for k in county_list[0:2]:
    #可再設定其他氣象站條件
    globals()['df_climate_'+str(county[k])] = pd.DataFrame()
    stations = list(st_info[st_info['城市']==k]['站號'])
    for i in os.listdir():
        df = pd.read_table(i,encoding='big5',comment='*',delimiter='\s+')
        col_names = df.columns[1:]
        df.drop(df.columns[-1:],axis=1,inplace=True)
        df.columns=col_names
        df = df[df['stno'].isin(stations)]
        globals()['df_climate_'+str(county[k])] = pd.concat([globals()['df_climate_'+str(county[k])],df],ignore_index=True)

In [65]:
county

{'嘉義縣': 0,
 '宜蘭縣': 1,
 '臺東縣': 2,
 '新北市': 3,
 '臺南市': 4,
 '雲林縣': 5,
 '花蓮縣': 6,
 '連江縣': 7,
 '彰化縣': 8,
 '金門縣': 9,
 '屏東縣': 10,
 '南投縣': 11,
 '臺北市': 12,
 '嘉義市': 13,
 '臺中市': 14,
 '新竹縣': 15,
 '新竹市': 16,
 '桃園市': 17,
 '高雄市': 18,
 '澎湖縣': 19,
 '苗栗縣': 20,
 '基隆市': 21}

In [70]:
climate_list

[         CD01  CD02  CD03  CD04  CD05  CD06  CD07  CD08  CD09  CD10  ...  \
 0         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
 1         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
 2         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
 3         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
 4         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
 ...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
 1232683   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
 1232684   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
 1232685   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
 1232686   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
 1232687   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
 
            WD01    WD02  WD03  WD04  WD05  WD06  WD07 WD08    stno  yyyym

In [82]:
df_climate_1

,CD01,CD02,CD03,CD04,CD05,CD06,CD07,CD08,CD09,CD10,...,WD04,WD05,WD06,WD07,WD08,stno,year,month,day,hour
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C0U520,2016,01,01,01
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C0U520,2016,01,01,02
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C0U520,2016,01,01,03
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C0U520,2016,01,01,04
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C0U520,2016,01,01,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C1U920,2020,12,31,20
1150580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C1U920,2020,12,31,21
1150581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C1U920,2020,12,31,22
1150582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C1U920,2020,12,31,23


### 資料整理

In [73]:
#建立氣象代碼轉換字典
col_dic = {'CD01':'高雲高度(Km)'
,'CD02':'中雲高度(Km)'
,'CD03':'低雲高度(Km)'
,'CD04':'高雲量(10分量)'
,'CD05':'中雲量(10分量)'
,'CD06':'低雲量(10分量)'
,'CD07':'高雲狀'
,'CD08':'中雲狀'
,'CD09':'低雲狀'
,'CD10':'雲冪高(Km)'
,'CD11':'總雲量(10分量)'
,'EP03':'A型蒸發量(mm)'
,'PP01':'降水量(mm)'
,'PP02':'降水時數(hr)'
,'PS01':'測站氣壓(hPa)'
,'PS02':'海平面氣壓(hPa)'
,'RH01':'相對濕度(%)'
,'RH02':'水氣壓(hPa)'
,'ST01':'天空狀況'
,'ST02':'地面狀況'
,'ST03':'雷暴'
,'ST04':'液態降水1'
,'ST05':'液態降水2'
,'ST06':'固態降水1'
,'ST07':'固態降水2'
,'ST08':'固態降水3'
,'ST09':'視障1'
,'ST10':'視障2'
,'ST11':'光象1'
,'ST12':'光象2'
,'SS01':'日照時數(hr)'
,'SS02':'全天空日射量(MJ/㎡)'
,'SS03':'淨輻射量(MJ/㎡)'
,'TS01':'地表0cm溫度(℃)'
,'TS02':'地中5cm溫度(℃)'
,'TS03':'地中10cm溫度(℃)'
,'TS04':'地中20cm溫度(℃)'
,'TS05':'地中30cm溫度(℃)'
,'TS06':'地中溫度50cm(℃)'
,'TS07':'地中溫度100cm(℃)'
,'TX01':'氣溫(℃)'
,'TX02':'最高氣溫(℃)'
,'TX03':'最低氣溫(℃)'
,'TX04':'濕球氣溫(℃)'
,'TX05':'露點溫度(℃)'
,'VS01':'能見度(Km)'
,'WD01':'平均風風速(m/s)'
,'WD02':'平均風風向(360 degree)'
,'WD03':'最大平均風風速(m/s)'
,'WD04':'最大平均風風向(360 degree)'
,'WD05':'最大瞬間風風速(m/s)'
,'WD06':'最大瞬間風風向(360 degree)'}

In [90]:
for k in county_list[0:2]:
    #時間點
    globals()['df_climate_'+str(county[k])]['year'] = globals()['df_climate_'+str(county[k])]['yyyymmddhh'].map(lambda x:str(x)[:4])
    globals()['df_climate_'+str(county[k])]['month'] = globals()['df_climate_'+str(county[k])]['yyyymmddhh'].map(lambda x:str(x)[4:6])
    globals()['df_climate_'+str(county[k])]['day'] = globals()['df_climate_'+str(county[k])]['yyyymmddhh'].map(lambda x:str(x)[6:8])
    globals()['df_climate_'+str(county[k])]['hour'] = globals()['df_climate_'+str(county[k])]['yyyymmddhh'].map(lambda x:str(x)[-2:])
    globals()['df_climate_'+str(county[k])].drop('yyyymmddhh',axis=1,inplace=True)
    #缺值改為空值
    globals()['df_climate_'+str(county[k])] = j.replace([-9991,-9996,-9997,-9998,-9999], np.nan)
    #欄位中文化
    globals()['df_climate_'+str(county[k])] = j.rename(columns=col_dic)

In [97]:
#groupby、aggregate計算需要的數值
df_climate_0.groupby(['year','month','day']).agg({'氣溫(℃)':['mean','max'],'平均風風速(m/s)':'max'})

氣溫(℃)       平均風風速(m/s)
                     mean   max        max
year month day                            
2016 01    01   15.296875  23.2        4.6
           02   18.467222  25.7        5.0
           03   17.685093  21.8        6.0
           04   19.630814  24.8        4.5
           05   19.504706  26.9       10.5
...                   ...   ...        ...
2020 12    27   12.606855  24.5        7.7
           28   11.231696  21.1       13.7
           29   13.919970  25.5       11.5
           30    4.956696  22.9       14.5
           31    1.377623  11.2        7.4

[1827 rows x 3 columns]